# Modeling Tool Prototype

Basic tool for building models using the `IFneuron` class.

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IFneuron import *

### NeuronNetwork Class

This class is used to create networks of `IFneuron` objects.

**Some notes to be aware of when using:**
- There is very little input validation, especially for adding connections. So be careful with `neuron_id` arguments
- You sort of have to run everything in the proper order. Start with adding neurons -> add connections -> set the stimulation times -> run the simulation -> collect the data.

In [ ]:
class NeuronNetwork:
    '''
    Data structure for storing networks of IFneurons.
    '''
    def __init__(self, id:'str'):
        self.id = id
        self.all_neurons = {}

    def add_neuron(self, neuron_id:str):
        neuron = IFneuron(neuron_id)
        if neuron in self.all_neurons:
            print(f'Neuron: {neuron.id} already exists. Use a different id.')
            return
        self.all_neurons[neuron_id] = neuron
    
    def add_neuron_connection(self, presynaptic_neuron_id:str, postsynaptic_neuron_id:str, weight:float):
        '''
        presynaptic -> postsynaptic.
        presynaptic_neuron_id is the 'sending' neuron.
        postsynaptic_neuron_id is the 'receiving' neuron.
        '''
        connection = (self.all_neurons[presynaptic_neuron_id], weight)
        self.all_neurons[postsynaptic_neuron_id].receptors.append(connection)
        # print(f'Connection formed: {presynaptic_neuron_id} -> {postsynaptic_neuron_id}')  # If you want to see whats happening

    def set_direct_stimulation_time_ms(self, neuron_id:str, stimulation_time_ms:list):
        self.all_neurons[neuron_id].t_directstim_ms = stimulation_time_ms
        # print(f'Set neuron: {neuron_id} stim times(ms) to: {self.all_neurons[neuron_id].t_directstim_ms}')

    def run_simulation(self, run_time_ms:float, record_membrane_potential:bool):
        for step in range(run_time_ms):
            for i in self.all_neurons.values():
                i.update(step, record_membrane_potential)

    def get_neuron_spike_times_ms(self, neuron_id:str) -> list:
        '''
        Returns individual neuron spike times(ms) as a list.
        '''
        return self.all_neurons[neuron_id].t_act_ms
    
    def get_neuron_membrane_potentials(self, neuron_id:str) -> list:
        '''
        Returns individual neuron membrane potentials as a list.
        '''
        return self.all_neurons[neuron_id].Vm_recorded
    
    def get_neuron_run_time_ms(self, neuron_id:str) -> list:
        '''
        Returns individual neuron run time(ms) as a list.
        This can be used to get the run time(ms) of the system.
        '''
        return self.all_neurons[neuron_id].t_recorded_ms

    def get_all_neurons(self) -> dict:
        '''
        Returns dictionary of all neurons in the system.
        Key: neuron_id,
        Value: IFneuron object
        '''
        return self.all_neurons
    
    def plot_membrane_potential_over_time(self, neuron_ids:list):
        '''
        neuron_ids is a list of ids.
        The ids can be pulled from get_all_neurons().
        '''
        fig, ax = plt.subplots(len(neuron_ids), figsize=(10, len(neuron_ids)*3), sharex=True, sharey=True)
        fig.suptitle('Membrane potential (mV) over Time (ms)')
        fig.supxlabel('Time (ms)')
        fig.supylabel('Membrane Potential (mV)')


    def print_all_neurons(self):
        print(self.all_neurons)

### Helper Functions

## Making a model

This example will use the original diagram, to keep things simple for now.

The diagram can be found here: https://docs.google.com/drawings/d/1Ff46J6rN11y0qpxOj1pYI5ceD3pyPDgKLzcZ0Jx5_uQ/edit?usp=sharing

### Creating the Ground Truth system

I chose to hold off on making a `generate_neurons` function for now so that we can better see what neurons are part of a system. This should be added later though, if we want to make example networks with random connections.

In [ ]:
ground_truth_system = NeuronNetwork('Ground Truth sim')

ground_truth_system.add_neuron('Neuron_A')  # input neuron
ground_truth_system.add_neuron('Neuron_B')  # input neuron
ground_truth_system.add_neuron('Neuron_C')
ground_truth_system.add_neuron('Neuron_D')
ground_truth_system.add_neuron('Neuron_E')
# ground_truth_system.print_all_neurons()

ground_truth_system.add_neuron_connection('Neuron_A', 'Neuron_C', .7)    # Neuron_A -> Neuron_C
ground_truth_system.add_neuron_connection('Neuron_B', 'Neuron_C', .7)    # Neuron_B -> Neuron_C

ground_truth_system.add_neuron_connection('Neuron_A', 'Neuron_D', 1.0)   # Neuron_A -> Neuron_D
ground_truth_system.add_neuron_connection('Neuron_B', 'Neuron_D', 1.0)   # Neuron_B -> Neuron_D

ground_truth_system.add_neuron_connection('Neuron_D', 'Neuron_E', 1.0)   # Neuron_D -> Neuron_E
ground_truth_system.add_neuron_connection('Neuron_C', 'Neuron_E', -1.0)  # Neuron_C -> Neuron_E

stim_time_ms_a = [25, 350, 700, 1000, 1400, 2000]
ground_truth_system.set_direct_stimulation_time_ms('Neuron_A', stim_time_ms_a)

stim_time_ms_b = [20, 450, 700, 1000, 1300, 2200]
ground_truth_system.set_direct_stimulation_time_ms('Neuron_B', stim_time_ms_b)

simulation_run_time_ms = 3500
ground_truth_system.run_simulation(simulation_run_time_ms, True)

## Demonstration of getters

`get_all_neurons`:

In [4]:
all_neurons = ground_truth_system.get_all_neurons()
for index, neuron_id in enumerate(all_neurons):
    print(f'index: {index}  |  neuron_id: {neuron_id}  |  neuron: {all_neurons[neuron_id]}')

index: 0  |  neuron_id: Neuron_A  |  neuron: <IFneuron.IFneuron object at 0x000001B3B6E10050>
index: 1  |  neuron_id: Neuron_B  |  neuron: <IFneuron.IFneuron object at 0x000001B3F67C2D50>
index: 2  |  neuron_id: Neuron_C  |  neuron: <IFneuron.IFneuron object at 0x000001B3B5D6DF90>
index: 3  |  neuron_id: Neuron_D  |  neuron: <IFneuron.IFneuron object at 0x000001B3F69DC050>
index: 4  |  neuron_id: Neuron_E  |  neuron: <IFneuron.IFneuron object at 0x000001B3B6E00050>


---
`get_neuron_spike_times_ms`:

In [5]:
neuron_A_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_A')
print(f'Neuron A spike times: {neuron_A_spike_times}')

neuron_B_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_B')
print(f'Neuron B spike times: {neuron_B_spike_times}')

neuron_C_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_C')
print(f'Neuron C spike times: {neuron_C_spike_times}')

neuron_D_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_D')
print(f'Neuron D spike times: {neuron_D_spike_times}')

neuron_E_spike_times = ground_truth_system.get_neuron_spike_times_ms('Neuron_E')
print(f'Neuron E spike times: {neuron_E_spike_times}')

Neuron A spike times: [25, 350, 700, 1000, 1400, 2000]
Neuron B spike times: [20, 450, 700, 1000, 1300, 2200]
Neuron C spike times: [27, 704, 1004]
Neuron D spike times: [26, 357, 458, 702, 1002, 1307, 1408, 2007, 2207]
Neuron E spike times: [364, 467, 1314, 1417, 2014, 2214]


---
`get_neuron_membrane_potentials`:

In [6]:
neuron_A_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_A')
print(f'Neuron A membrane_potentials: {neuron_A_membrane_potentials[:10]}...')

neuron_B_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_B')
print(f'Neuron B membrane_potentials: {neuron_B_membrane_potentials[:10]}...')

neuron_C_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_C')
print(f'Neuron C membrane_potentials: {neuron_C_membrane_potentials[:10]}...')

neuron_D_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_D')
print(f'Neuron D membrane_potentials: {neuron_D_membrane_potentials[:10]}...')

neuron_E_membrane_potentials = ground_truth_system.get_neuron_membrane_potentials('Neuron_E')
print(f'Neuron E membrane_potentials: {neuron_E_membrane_potentials[:10]}...')

Neuron A membrane_potentials: [-60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0]...
Neuron B membrane_potentials: [-60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0]...
Neuron C membrane_potentials: [-60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0]...
Neuron D membrane_potentials: [-60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0]...
Neuron E membrane_potentials: [-60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0, -60.0]...


---
`get_neuron_run_time_ms`:

In [7]:
neuron_A_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_A')
print(f'Neuron A spike times: {neuron_A_run_time[:10]}...')

neuron_B_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_B')
print(f'Neuron B spike times: {neuron_B_run_time[:10]}...')

neuron_C_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_C')
print(f'Neuron C spike times: {neuron_C_run_time[:10]}...')

neuron_D_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_D')
print(f'Neuron D spike times: {neuron_D_run_time[:10]}...')

neuron_E_run_time = ground_truth_system.get_neuron_run_time_ms('Neuron_E')
print(f'Neuron E spike times: {neuron_E_run_time[:10]}...')

Neuron A spike times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
Neuron B spike times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
Neuron C spike times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
Neuron D spike times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
Neuron E spike times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
